In [133]:
"""
Ejercicio de Regresión para Entrevista de Trabajo
Fecha: 7 de Junio de 2024
Autor: Rodolfo Abraham Sánchez Isidro
Propósito: Este script entrena un modelo de regresión de árbol de decisión para predecir la tasa de abandono en una empresa de comercio electrónico y muestra la importancia de las características.

"""

# Importar bibliotecas necesarias
import pandas as pd
from sklearn.metrics import mean_absolute_error, r2_score
from sklearn.tree import DecisionTreeRegressor
from sklearn.model_selection import train_test_split
from sklearn.impute import SimpleImputer
from sklearn.ensemble import RandomForestRegressor

# Cargar el archivo de Excel
file_path = '/Users/Abraham/iCloud Drive (Archive)/Documents/ML-Python/E_Commerce_Dataset.xlsx'
with pd.ExcelFile(file_path) as xls:
    Ecomm = pd.read_excel(xls, sheet_name='E Comm')

# Selección de características
y = Ecomm['Churn']

# Eliminar las columnas 'CustomerID' y 'Churn' del conjunto de características
features_to_encode = ['PreferredLoginDevice', 'CityTier', 'PreferredPaymentMode', 'Gender',
                      'PreferedOrderCat', 'MaritalStatus']

# Selección de características numéricas
numeric_features = ['Tenure', 'WarehouseToHome', 'HourSpendOnApp', 'NumberOfDeviceRegistered',
                    'SatisfactionScore', 'NumberOfAddress', 'Complain', 'OrderAmountHikeFromlastYear',
                    'CouponUsed', 'OrderCount', 'DaySinceLastOrder', 'CashbackAmount']

# Obtener las características numéricas
X_numeric = Ecomm[numeric_features]

# Obtener las características categóricas y realizar codificación one-hot
X_categorical = pd.get_dummies(Ecomm[features_to_encode])

# Combinar características numéricas y categóricas codificadas one-hot
X_encoded = pd.concat([X_numeric, X_categorical], axis=1)

# Imputar los valores faltantes
imputer = SimpleImputer(strategy='mean')
X_imputed = imputer.fit_transform(X_encoded)

# Dividir los datos en conjuntos de entrenamiento y validación
train_X, val_X, train_y, val_y = train_test_split(X_imputed, y, random_state=0)

# Crear y entrenar el modelo
Ecomm_model = RandomForestRegressor(max_leaf_nodes=1000, random_state=0)
Ecomm_model.fit(train_X, train_y)

# Hacer predicciones en el conjunto de validación
preds_all_data = Ecomm_model.predict(val_X)

# Calcular el error absoluto medio (MAE)
mae = mean_absolute_error(val_y, preds_all_data)

# Calcular el coeficiente de determinación R^2
r2 = r2_score(val_y, preds_all_data)

# Mostrar los resultados
print("Resultados del Modelo:")
print("---------------------")
print(f"Error absoluto medio (MAE): {mae:.2f}")
print(f"Coeficiente de determinación R^2: {r2:.2f}")
print("\nImportancia de las Características:")
print("----------------------------------")
print(pd.DataFrame({'Feature': X_encoded.columns, 'Importance': Ecomm_model.feature_importances_}))


Resultados del Modelo:
---------------------
Error absoluto medio (MAE): 0.10
Coeficiente de determinación R^2: 0.73

Importancia de las Características:
----------------------------------
                                  Feature  Importance
0                                  Tenure    0.279948
1                         WarehouseToHome    0.067608
2                          HourSpendOnApp    0.012568
3                NumberOfDeviceRegistered    0.029743
4                       SatisfactionScore    0.043061
5                         NumberOfAddress    0.069723
6                                Complain    0.064009
7             OrderAmountHikeFromlastYear    0.047763
8                              CouponUsed    0.021399
9                              OrderCount    0.018212
10                      DaySinceLastOrder    0.067477
11                         CashbackAmount    0.101516
12                               CityTier    0.022172
13          PreferredLoginDevice_Computer    0.014059
1

In [138]:
from sklearn.metrics import mean_absolute_error
from sklearn.tree import DecisionTreeRegressor

def get_mae(max_leaf_nodes, train_X, val_X, train_y, val_y):
    model = RandomForestRegressor(max_leaf_nodes=max_leaf_nodes, random_state=0)
    model.fit(train_X, train_y)
    preds_val = model.predict(val_X)
    mae = mean_absolute_error(val_y, preds_val)
    return(mae)

In [139]:
# compare MAE with differing values of max_leaf_nodes
for max_leaf_nodes in [2,5, 50, 500, 5000, 10000]:
    my_mae = get_mae(max_leaf_nodes, train_X, val_X, train_y, val_y)
    print("Max leaf nodes: %f  \t\t Mean Absolute Error:  %f" %(max_leaf_nodes, my_mae))

Max leaf nodes: 2.000000  		 Mean Absolute Error:  0.218042
Max leaf nodes: 5.000000  		 Mean Absolute Error:  0.197000
Max leaf nodes: 50.000000  		 Mean Absolute Error:  0.146128
Max leaf nodes: 500.000000  		 Mean Absolute Error:  0.095788
Max leaf nodes: 5000.000000  		 Mean Absolute Error:  0.095788
Max leaf nodes: 10000.000000  		 Mean Absolute Error:  0.095788


In [140]:
from sklearn.model_selection import cross_val_score

# Validación cruzada para Random Forest
cv_scores = cross_val_score(Ecomm_model, X_imputed, y, cv=5, scoring='r2')
print(f"Scores de validación cruzada: {cv_scores}")
print(f"Promedio de R^2 en validación cruzada: {cv_scores.mean():.2f}")


Scores de validación cruzada: [0.89431804 0.84685016 0.88887594 0.86972957 0.88297646]
Promedio de R^2 en validación cruzada: 0.88
